In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.\
        builder.\
        appName('Transformation').\
        master('spark://DOUGPC.:7077').\
        getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/05/21 09:18:51 WARN Utils: Your hostname, DOUGPC resolves to a loopback address: 127.0.1.1; using 172.28.170.236 instead (on interface eth0)
23/05/21 09:18:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/21 09:18:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/21 09:18:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/21 09:18:53 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master DOUGPC.:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:515)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:264)
	at java.base/java.util.concurrent.Thre

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/doug/.local/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/doug/.local/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

23/05/21 09:19:53 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: All masters are unresponsive! Giving up.
23/05/21 09:19:53 WARN StandaloneSchedulerBackend: Application ID is not initialized yet.
23/05/21 09:19:53 WARN StandaloneAppClient$ClientEndpoint: Drop UnregisterApplication(null) because has not yet connected to master
23/05/21 09:19:53 ERROR AsyncEventQueue: Listener AppStatusListener threw an exception
java.lang.NullPointerException
	at org.apache.spark.status.AppStatusListener.onApplicationEnd(AppStatusListener.scala:192)
	at org.apache.spark.scheduler.SparkListenerBus.doPostEvent(SparkListenerBus.scala:57)
	at org.apache.spark.scheduler.SparkListenerBus.doPostEvent$(SparkListenerBus.scala:28)
	at org.apache.spark.scheduler.AsyncEventQueue.doPostEvent(AsyncEventQueue.scala:37)
	at org.apache.spark.scheduler.AsyncEventQueue.doPostEvent(AsyncEventQueue.scala:37)
	at org.apache.spark.util.ListenerBus.postToAll(ListenerBus.scala:117)
	at org.apache.spark.u

In [ ]:
schemaFrame  = StructType([StructField('InvoiceNo', StringType(), True),
                          StructField('StockCode', StringType(), True),
                          StructField('Description', StringType(), True),
                          StructField('Quantity', IntegerType(), True),
                          StructField('InvoiceDate', StringType(), True),
                          StructField('UnitPrice', DecimalType(5,2), True),
                          StructField('CustomerID', StringType(), True),
                          StructField('Country', StringType(), True)])

retaildataFrame = spark.read.csv('../Python/SparkDefGuide/data/retail-data/all/online-retail-dataset.csv', header= True, schema= schemaFrame)

flightFrame = spark.read.csv('/home/doug/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv', header= True)

retaildataFrame.show(2)

flightFrame.show(2)

In [ ]:
# retaildataFrame.show()

retailStudy = retaildataFrame.selectExpr('Quantity', 'UnitPrice',\
                                            'Quantity * 2 as DoubleQtd').\
                            show(5)

In [ ]:
retaildataFrame.select(
                        retaildataFrame.Quantity,\
                        retaildataFrame.UnitPrice,\
                        (retaildataFrame.Quantity * retaildataFrame.UnitPrice).\
                            alias('Total').\
                            cast("float")).\
                show()

In [ ]:
flydataframe2012 = spark.read.csv('../Python/SparkDefGuide/data/flight-data/csv/2012-summary.csv', header= True).\
                        withColumnRenamed('ORIGIN_COUNTRY_NAME','ORIGIN').\
                        withColumnRenamed('DEST_COUNTRY_NAME', 'DESTINE').\
                        withColumnRenamed('count', 'FLY_QTD')

flydataframe2011 = spark.read.csv('../Python/SparkDefGuide/data/flight-data/csv/2011-summary.csv', header= True).\
                        withColumnRenamed('ORIGIN_COUNTRY_NAME','ORIGIN').\
                        withColumnRenamed('DEST_COUNTRY_NAME', 'DESTINE').\
                        withColumnRenamed('count', 'FLY_QTD')
                        
print(flydataframe2011.count())

print(flydataframe2012.count())

In [ ]:
#UNINDO OS DATAFRAMES

#union dataframe
#anexa um dataframe ao outro
#como na linguagem SQL, precisam ter o mesmo schema e numero de colunas
#nome não faz diferença

flydataframe2011.union(flydataframe2012).count()

#COUNT DISTINCT

flydataframe2011.select("ORIGIN", "FLY_QTD").\
                    where(flydataframe2011.FLY_QTD > 500).\
                    agg(countDistinct('ORIGIN').\
                        alias('DIST_ORIGIN')).show() # type: ignore

In [ ]:
fnewframe2011=spark.read.csv('../Python/SparkDefGuide/data/flight-data/csv/2011-summary.csv', header= True).\
                        withColumnRenamed('ORIGIN_COUNTRY_NAME','ORIGIN').\
                        withColumnRenamed('DEST_COUNTRY_NAME', 'DESTINE').\
                        withColumnRenamed('count', 'FLY_QTD').\
                    show(2)

In [ ]:
retaildataFrame.withColumn('Price_10%',
                            format_number((retaildataFrame.UnitPrice * 0.10 + retaildataFrame.UnitPrice), 2)).\
                show(5)

In [ ]:
retaildataFrame.select('UnitPrice').repartition(3, 'UnitPrice').show()

In [ ]:
# retailFrame2012 = retaildataFrame.\
#                     filter("InvoiceDate BETWEEN '1/01/2010 AND '31/12/2010'").\
#                     select('CustomerID', 'InvoiceDate', 'UnitPrice', 'Quantity').\
#                     orderBy('InvoiceDate', ascending=True).\
#                     show(1000)

                            
df = retaildataFrame.filter((col("invoiceDate")>= '01/01/2010') & (col("InvoiceDate") <= '31/12/2010')).\
                        select("InvoiceDate", 'CustomerId', 'Unitprice', "Quantity").\
                        explain(True)